# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import pandas as pd

In [ ]:
def get_response(request_url):
    headers =({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})
    return requests.get(request_url, headers = headers)

In [ ]:
def get_elements_by_attributes(source_data, tag, attributes):
    scrapped_data = BeautifulSoup(source_data, "html.parser")
    return scrapped_data.find_all(tag, attributes)

In [ ]:
def get_elements_by_tag(source_data, tag):
    scrapped_data = BeautifulSoup(source_data, "html.parser")
    return scrapped_data.find_all(tag)

In [ ]:
# Your code here
# A) Amazon Product - https://www.amazon.com/Apple-MacBook-13-inch-256GB-Storage/
# Using webscrapping the data is being fetched and stored
# URL - https://www.amazon.com/Apple-MacBook-13-inch-256GB-Storage/product-reviews/B08N5M7S6K/ref=cm_cr_getr_d_paging_btm_prev_1?pageNumber=1

product_url = "https://www.amazon.com/Apple-MacBook-13-inch-256GB-Storage/product-reviews/B08N5M7S6K/ref=cm_cr_getr_d_paging_btm_prev_{}?pageNumber={}"

nextPage, current_page = True, 1
reviews = []
while(nextPage and len(reviews) < 1000) :
    response = get_response(product_url).text
    page_content = BeautifulSoup(response.content, "html.parser")
    print(page_content)
    reviews = page_content.find_all("span", {"data-hook" : "review-body"})
    print(len(reviews))

In [ ]:
movie_review_url = "https://www.imdb.com/title/tt13751694/reviews?ref_=tt_urv"
reviews = []
while(len(reviews) < 1000):
    response_data = get_response(movie_review_url).text
    elements = get_elements_by_attributes(response_data, "div", {"class" : "text show-more__control"})
    pagination_key = get_elements_by_attributes(response_data, 'div',{'class' : 'load-more-data'})[0]["data-key"]
    reviews.extend([{"s.no" : len(reviews) + 1, "review" : i.text} for i, j in zip(elements, range(1, len(elements) + 1))])
    movie_review_url = "https://www.imdb.com/title/tt13751694/reviews/_ajax?&paginationKey="+pagination_key
movie_reviews_df = pd.json_normalize(reviews)
movie_reviews_df.to_csv("movie_reviews.csv")

In [ ]:
capterra_url = "https://www.capterra.com/customer-relationship-management-software/?sort=most_reviews?page={}"
product, page = 0, 2
while(product < 1000):
    print(capterra_url.format(page))
    response = get_response(capterra_url.format(page)).text
    print(response)
    products = get_elements_by_attributes(response, "a", {"data-evt-name" : "engagement_product_click"})
    print(products)
    break

In [ ]:
import json

semantic_scholar_url = "https://api.semanticscholar.org/graph/v1/paper/search?query={}&offset={}&limit={}&fields=abstract"
keyword = "artifical intelligence"
abstracts, offset = [], 0
while(len(abstracts) < 10000):
    print(semantic_scholar_url.format(keyword, offset, 100))
    response = get_response(semantic_scholar_url.format(keyword, offset, 100)).text
    if("data" in response):
        offset += 100
        abstracts.extend(json.loads(response).get("data", []))
    print(len(abstracts))
print(abstracts)

In [ ]:
ddr_url = "https://ddr.densho.org/narrators/{}/"
infos, pages = [], 0
while(pages < 904):
    response = get_response(ddr_url.format(page))
    if(response.status_code != 200):
        continue
    infos.append({"s.no" : page, "info" : get_elements_by_tag(response.text, "p")[0].text})
    pages += 1
narrators_df = pd.json_normalize(infos)
narrators_df.to_csv("narrators_info.csv")

# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [ ]:
# Write code for each of the sub parts with proper comments.

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

class Processor:

    ps = PorterStemmer()
    ltzr = WordNetLemmatizer()
    stopwords = set(stopwords.words('english'))

    @classmethod
    def clean_text(cls, data):

        # Remove special characters
        data = re.sub(r'[^\w\s]', '', data)

        # Remove Digits
        data = re.sub(r'\d+', '', data)

        # Tokenization
        tokens = nltk.word_tokenize(data.lower())

        # Stopwords Removal
        tokens = [token for token in tokens if token not in cls.stopwords]

        # Stemming
        tokens = [cls.ps.stem(token) for token in tokens]

        # Lemmitization
        tokens = [cls.ltzr.lemmatize(token) for token in tokens]

        # Joining the tokens together and returning it.
        return ' '.join(tokens)

    @classmethod
    def process_data(cls, file_name):
        df = pd.read_csv(file_name)
        print(f"Read file - {file_name}")
        columns = df.columns
        new_col = f"Cleaned_{columns[2]}"
        df[new_col] = df[columns[2]].apply(cls.clean_text)
        df.to_csv(f"Cleaned_{file_name}")

file_names = ["movie_reviews.csv", "narrators_info.csv"]
for file_name in file_names:
    Processor.process_data(file_name)

# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
! pip install stanfordnlp

In [ ]:
# Your code here
import stanfordnlp

stanfordnlp.download('en', force=True)

nlp = stanfordnlp.Pipeline()

def display_constituency_tree(sentence):

    parsed = nlp(sentence)
    tree_str = parsed.sentences[0].parse_tree
    tree = Tree.fromstring(tree_str)
    tree.pretty_print()

def display_dependency_tree(sentence):

    parsed = nlp(sentence)
    print(parsed.sentences[0].dependencies)

display_constituency_tree("anim classic bollywood garbagerif misogyni senseless plot twist film lack cohes storylin drown sea unnecessari turn what truli entertain sift suspici rate becom game spot paid botgener review make detect work enjoy endur movi realm bollywood disappoint movi serv remind challeng bollywood filmmak lacklust plot abund unnecessari twist distress dose misogyni overshadow cinemat potenti")
display_dependency_tree("print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.")

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:
# Write your response below